In [1]:
import pandas as pd
import selenium
import time
import re
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
from selenium.webdriver.chrome.options import Options

In [2]:
chrome_options = Options()

# incognito window
#chrome_options.add_argument("--incognito")

# open a webbrowser to scrape
driver = webdriver.Chrome(r'C:\Users\AKHIL\Downloads\chromedriver_win32\chromedriver.exe')

In [3]:
url = 'https://www.cars24.com/buy-used-car?sort=P&storeCityId=2&pinId=110001'
driver.get(url)
driver.maximize_window()
driver.switch_to.window(driver.window_handles[0])

In [4]:
place_elm = driver.find_element_by_xpath("//p[@class='_1OR7d']")
place_elm.click()

In [5]:
location_elm = driver.find_elements_by_xpath("//ul[@class='_16Bvy']//li//h6")

In [6]:
length = len(location_elm)

In [7]:
location = location_elm[0].text
print(location)
location_elm[0].click()

New Delhi


In [8]:
# creating a dataframe with the required columns
columns = ["Location","Type","Brand","Model","Variant","Year","Kilometers","No_of_owners","Fuel","Transmission","Price"]
df = pd.DataFrame(columns=columns)

In [9]:
#scraping data
for i in range(length):
    
    place_elm = driver.find_element_by_xpath("//p[@class='_1OR7d']")
    place_elm.click()
    
    time.sleep(1)
    
    location_elm = driver.find_elements_by_xpath("//ul[@class='_16Bvy']//li//h6")
    location_click_elm = driver.find_elements_by_xpath("//ul[@class='_16Bvy']//li//span")
    location = location_elm[i].text
    location_click_elm[i].click()
    
    time.sleep(1)
    filter_elm = driver.find_element_by_xpath("//ul[@class='_1bWR_']//li[@class='ykp-v'][6]")
    filter_elm.click()
    
    filter_elm1 = driver.find_elements_by_xpath("//ul[@class='_1bWR_']//li[@class='ykp-v'][6]//div[@class='filter-section _1PX-6 _2IgOS']//div[1]//div[3]//ul//li")
    filter_elm.click()
    for f in range(0,len(filter_elm1)):
        filter_elm = driver.find_element_by_xpath("//ul[@class='_1bWR_']//li[@class='ykp-v'][6]")
        filter_elm.click()

        time.sleep(1)
        scroll_elm = driver.find_element_by_xpath("//div[@class='dJM1f']")
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_elm)

        filter_elm1 = driver.find_elements_by_xpath("//ul[@class='_1bWR_']//li[@class='ykp-v'][6]//div[@class='filter-section _1PX-6 _2IgOS']//div[1]//div[3]//ul//li")
        time.sleep(1)

        Type = filter_elm1[f].text
        filter_elm1[f].click()
        
        time.sleep(1)
        search_btn = driver.find_element_by_xpath("//ul[@class='_1bWR_']//li[@class='ykp-v'][6]//div[@class='filter-section _1PX-6 _2IgOS']//div[2]//button")
        search_btn.click()
        
        for s in range(1,5):
            driver.execute_script("window.scrollTo(0 ,0);")
            driver.execute_script("window.scrollTo(0 ,document.body.scrollHeight);")
            time.sleep(1)
            
        link_elm = driver.find_elements_by_xpath("//div[@class='col-4']//a")
        links = [i.get_attribute("href") for i in link_elm]
        
        driver.execute_script("window.open();")
        driver.switch_to.window(driver.window_handles[-1])
        
        for l in range(len(links)):
            driver.get(links[l])
            time.sleep(1)
            
            try:
                car_elm = driver.find_element_by_xpath("//div[@class='_187DF']//h2")
                variant_elm = driver.find_element_by_xpath("//div[@class='_187DF']//div[@class='_10kZS']//h2//span")
            
                year = car_elm.text.split(" ")[0]
                brand = car_elm.text.split(" ")[1]
                variant = variant_elm.text.replace("Manual",'').replace("Automatic",'').replace("MANUAL",'').replace("AUTOMATIC",'').strip()
            
                model = car_elm.text.replace(brand,'').replace(year,'').replace(variant,'')
                model = re.sub(r'\s+',' ',model)
                model = model.strip()
            
                features_elm = driver.find_elements_by_xpath("//ul[@class='_2KO13']//li//label")
                len(features_elm)
                features_value = driver.find_elements_by_xpath("//ul[@class='_2KO13']//li//strong")
                len(features_value)
            
                for i in range(len(features_elm)):
                    if features_elm[i].text == "Kilometers Driven":
                        t = features_value[i].text
                        t = re.sub("\D","",t)
                        t = re.sub(r'\s+',' ',t)
                        Kilometers = t.strip()
                    else:
                        if features_elm[i].text == "Owner":
                            t = features_value[i].text
                            t = re.sub("\D","",t)
                            t = re.sub(r'\s+',' ',t)
                            No_of_owners = t.strip()
                        else:
                            if features_elm[i].text == "Fuel Type":
                                Fuel = features_value[i].text
                            else:
                                if features_elm[i].text == "Transmission":
                                    Transmission = features_value[i].text
                                
                try:
                    price_elm = driver.find_element_by_xpath("//div[@class='_187DF']//div[@class='text-right g-hPT']//strong//span")
                except:
                    price_elm = driver.find_element_by_xpath("//div[@class='_187DF']//div[@class='text-right g-hPT']//strong")
                
                t = price_elm.text
                t = re.sub("\D","",t)
                t = re.sub(r'\s+',' ',t)
                Price = t.strip()
            
                row = [location,Type,brand,model,variant,year,Kilometers,No_of_owners,Fuel,Transmission,Price]
                df = pd.concat([df,pd.DataFrame([row], columns = columns)],ignore_index = True)
                
            except:
                pass
        
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
        driver.execute_script("window.scrollTo(0 ,0);")
        
        try:
            clear_elm = driver.find_element_by_xpath("//div[@class='col-md-10 d-flex align-items-center']//div//ul")
            clear_elm.click()
        except:
            try:
                filter_elm = driver.find_element_by_xpath("//ul[@class='_1bWR_']//li[@class='ykp-v'][6]")
                filter_elm.click()

                time.sleep(1)
                scroll_elm = driver.find_element_by_xpath("//div[@class='dJM1f']")
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scroll_elm)
            
                filter_elm1 = driver.find_elements_by_xpath("//ul[@class='_1bWR_']//li[@class='ykp-v'][6]//div[@class='filter-section _1PX-6 _2IgOS']//div[1]//div[3]//ul//li")
                time.sleep(1)
                filter_elm1[f].click()
        
                filter_elm = driver.find_element_by_xpath("//ul[@class='_1bWR_']//li[@class='ykp-v'][6]")
                filter_elm.click()
            except:
                break
        
driver.close()

In [10]:
print(df.shape)
df.sample(10)

(5126, 11)


,Location,Type,Brand,Model,Variant,Year,Kilometers,No_of_owners,Fuel,Transmission,Price
1164,Noida,Luxury SUV,Toyota,Fortuner AUTOMATIC,2.8 4x4 AT,2017,21670,1,Diesel,AUTOMATIC,2958299
220,New Delhi,Hatchback,Renault,Kwid MANUAL,RXT,2016,46028,2,Petrol,MANUAL,280099
3335,Pune,Luxury Sedan,Toyota,Corolla Altis MANUAL,G,2013,71052,3,Petrol,MANUAL,407299
2824,Pune,Hatchback,Maruti,Alto 800 MANUAL,LXI,2017,15483,1,Petrol,MANUAL,340399
681,Noida,Hatchback,Maruti,Alto K10 MANUAL,VXI,2013,17374,1,Petrol,MANUAL,257999
3157,Pune,SUV,Maruti,Vitara Brezza MANUAL,VDI AMT,2018,34666,1,Diesel,MANUAL,914499
4369,Chennai,SUV,Maruti,Eeco MANUAL,5 STR WITH AC PLUSHTR,2020,13035,1,Petrol,MANUAL,424799
1655,Gurgaon,SUV,Hyundai,Creta MANUAL,1.4 E PLUS CRDI,2019,54598,1,Diesel,MANUAL,980000
694,Noida,Hatchback,Maruti,Alto K10 MANUAL,VXI,2017,11691,1,Petrol,MANUAL,381599
4314,Chennai,Sedan,Maruti,Swift Dzire MANUAL,VDI BS IV,2014,83907,1,Diesel,MANUAL,482899


In [11]:
df.drop_duplicates(inplace=True,ignore_index=True)
print(df.shape)

(5045, 11)


In [12]:
df.tail()

,Location,Type,Brand,Model,Variant,Year,Kilometers,No_of_owners,Fuel,Transmission,Price
5040,Ahmedabad,Luxury SUV,Landrover,Range Rover Evoque AUTOMATIC,DYNAMIC SD4,2015,129752,2,Diesel,AUTOMATIC,1765499
5041,Ahmedabad,Luxury SUV,Jeep,Compass MANUAL,2.0 LIMITED 4*2,2017,148933,1,Diesel,MANUAL,1203899
5042,Ahmedabad,Luxury SUV,Toyota,Fortuner MANUAL,3.0 MT 4X4,2012,184782,3,Diesel,MANUAL,1251799
5043,Ahmedabad,Luxury SUV,Mahindra,XUV500 MANUAL,W10,2018,69977,1,Diesel,MANUAL,1170499
5044,Ahmedabad,Luxury SUV,Mahindra,XUV500 MANUAL,W5 FWD,2018,35913,1,Diesel,MANUAL,829099


In [13]:
df.isnull().sum()

Location        0
Type            0
Brand           0
Model           0
Variant         0
Year            0
Kilometers      0
No_of_owners    0
Fuel            0
Transmission    0
Price           0
dtype: int64

In [14]:
df.to_csv("cars-data-scrape.csv")